In [1]:
library(repr)
library(dplyr)
library(GGally)
library(tidymodels)
library(leaps)
install.packages("fastDummies")
library(fastDummies)



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: ggplot2

Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2

── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tibble       3.2.1
✔ infer        1.0.5     ✔ tidyr        1.3.0
✔ modeldata    1.2.0     ✔ tune         1.1.2
✔ parsnip      1.1.1     ✔ workflows    1.1.3
✔ purrr        1.0.2     ✔ workflowsets 1.0.1
✔ recipes      1.0.8     ✔ yardstick    1.2.0

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ purrr::discard() masks scales::discard()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
✖ recipes::step()  masks stats::step()
• Search for functions across packag

## Introduction:
This dataset contains the count of rental bikes between the years 2011 and 2012 in Washington, D.C., USA for 2 years (2011-2012). This data is from the UCI Machine Learning Repository and collected by the Capital bike share system. There are 731 rows and 15 columns in our data. Bike-sharing system has a important role in traffic, environmental and health issues. Based on these data, we can also detect most of important events in the city. 

link to the source of dataset: https://archive.ics.uci.edu/dataset/275/bike+sharing+dataset <br>
link to the dataset on github: https://github.com/Yuji03b/toy_ds_project/blob/main/day.csv

## 1. Research Question

We want to investigate that, to what extent number of registered users of the bike-share system relates to a set of variables inculding feeling temperature, seasons, workingday and wind speed. We will focus on inference and see how the changes in these variables will affect the number of registered users of the bike-sharing system. By exploring pattern of the registered users, the result can aid in decision-making related to resource allocation, budget planning, and infrastructure development. 

In [2]:
bike_dat <- read.csv("https://raw.githubusercontent.com/Yuji03b/toy_ds_project/main/day.csv", row.names = 1, stringsAsFactors= TRUE)

## 2. The description of variables 
*The dataset contains 731 rows and 15 columns, where each column represents a distinct variable. Among these variables, `Registered` serves as the response variable while `atemp, season, weekday `and `windspeed` are the explanatory variables. Also, there are no missing values in the dataset.*

- **dteday** [factor variable] : date
- **season** [integer variable]: season (1:springer, 2:summer, 3:fall, 4:winter)
- **yr** [integer variable]: year (0: 2011, 1:2012)
- **mnth** [integer variable]: month ( 1 to 12)
- **hr** [integer variable] : hour (0 to 23)
- **holiday** [integer variable]: weather day is holiday or not (extracted from http://dchr.dc.gov/page/holiday-schedule)
- **weekday** [integer variable]: day of the week
- **workingday** [integer variable]: if day is neither weekend nor holiday is 1, otherwise is 0.
- **weathersit** [integer variable] : 
	 - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
	 - 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
	- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
	- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
- **temp** [double variable]: Normalized temperature in Celsius. The values are divided to 41 (max)
- **atemp** [double variable]: Normalized feeling temperature in Celsius. The values are divided to 50 (max)
- **hum** [double variable]: Normalized humidity. The values are divided to 100 (max)
- **windspeed** [double variable]: Normalized wind speed. The values are divided to 67 (max)
- **casual** [integer variable]: count of casual users
- **registered** [integer variable]: count of registered users
- **cnt** [integer variable]: count of total rental bikes including both casual and registered

## 3. Cleaning and Wrangling Data

`Step 1.` Some varibales will be rename to make them more descriptive.<br>
`Step 2.` Remove the variables that are not relevant to our analysis from the dataset.<br>
`Step 3.` All continuous variables have been scaled by "max scaling" method, ensuring that their values fall within the standardized range of 0 to 1. <br>
`Step 4.`There might be certain highly correlated variables, such as  `temperature` and `feeling temperature`, as well as `number registered user` and `count of total rental bikes`. Thus, we will examine ther correlation and remove the variables that might caused multicollinearity or serve as confounding factors.<br>


In [3]:
#step 1 rename the variables
colnames(bike_dat)[3] <- "year" 
colnames(bike_dat)[10] <- "feeling_temp" 
colnames(bike_dat)[11] <- "humidity" 
colnames(bike_dat)[14] <- "registered_users" 
colnames(bike_dat)[15] <- "total_rental_bike" 

In [4]:
#step 2  Remove the irrelevant variables 
bike_dat_step2<- select (bike_dat,-c(dteday,casual,mnth)) 
head(bike_dat_step2)


,season,year,holiday,weekday,workingday,weathersit,temp,feeling_temp,humidity,windspeed,registered_users,total_rental_bike
,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
1,1,0,0,6,0,2,0.344167,0.363625,0.805833,0.1604460,654,985
2,1,0,0,0,0,2,0.363478,0.353739,0.696087,0.2485390,670,801
3,1,0,0,1,1,1,0.196364,0.189405,0.437273,0.2483090,1229,1349
4,1,0,0,2,1,1,0.200000,0.212122,0.590435,0.1602960,1454,1562
5,1,0,0,3,1,1,0.226957,0.229270,0.436957,0.1869000,1518,1600
6,1,0,0,4,1,1,0.204348,0.233209,0.518261,0.0895652,1518,1606


In [5]:
#step 4 Examine ther correlation 
options(repr.plot.height = 17, repr.plot.width = 17)
step1_ggpair_plot <- bike_dat_step2 %>%
  ggpairs(progress = FALSE) +
  theme(
    text = element_text(size = 15),
    plot.title = element_text(face = "bold"),
    axis.title = element_text(face = "bold")
  )



The correlation table reveals a strong correlation between `temp` and `feeling_temp`, as well as between `registered_users` and `total_rental_bike`. Both of these correlations exceed 0.9. Since `total_rental_bike` is the sum of casual and registered users and cannot be served as an explanatory variable, it will be removed from the dataset. 

In [6]:
#step 4 Examine ther correlation and remove the variables
bike_dat_step4 <- select(bike_dat_step2,-c(temp,total_rental_bike,weekday,weathersit)) 

##  4.  Visualization

In [7]:
options(repr.plot.height = 10, repr.plot.width = 10)

user_ggpair_plot <- bike_dat_step4 %>%
  ggpairs(progress = FALSE) +
  theme(
    text = element_text(size = 15),
    plot.title = element_text(face = "bold"),
    axis.title = element_text(face = "bold")
  )



Based on the visualization plot, we can observe that "registered_user" as the response variable exhibits correlations with several variables. Notably, it shows a positive correlation with "season" (approximately 0.412), "workingday" (0.304), and "feeling_temp" (0.544), while it has a negative correlation with "windspeed" (around -0.217). This suggests that the number of registered users tends to increase with higher feeling temperature and decrease as windspeed increase. Also, all four of these correlations exceed 0.1, which is a notable value and indicates that these variables are meaningful and should be considered as explanatory factors in our analysis.

#### 1. season
 "Season" is a categorical variable with four levels, denoted by integers from 1 to 4, each signifying one of the four seasons. From the scatter plot in row 5, column 1, it indicates that there are more registered users in fall and winter compared to spring and summer.
 
Additionally, we observe a 0.343 correlation between "feeling_temp" and "season". This correlation is logical as temperature changes with the seasons. Consequently, when fitting the model, it is necessary to consider these two variables to avoid multicollinearity. To determine the most suitable model, we plan to compare the model with both `season` and `feeling_temp` to models excluding one of these variables. We will assess their R-squared values to determine which model best fits the data. We can also assess their respective p-values in the original model to determine their significance.<br>


#### 2. Working day
"Workingday" is another categorical variable within the dataset. In this context, if a day is neither a weekend nor a holiday, it is assigned the value 1; otherwise, it is assigned the value 0. The scatter plot in row 5, column 2, suggests that more people register with the bike-sharing system on workdays. We initially represent this variable as an integer for creating more legible plots. However, before fitting the model, we will transform it into a factor type. 

## 5. Method Part


We will divide the entire dataset into two parts using `initial_split` function. The training dataset will be used for selecting variables for the regression model, while the testing dataset will be reserved for inference purposes. This method is used to avoid an increase in Type 1 error, which can occur if the same data sample is used for both variable selection and model fitting. Given that there are 731 observations in the dataset, we will split it using a proportion `(prop)` of 0.5. Consequently, the training and testing datasets will contain 364 and 367 observations respectively.

Our dataset contains four explanatory variables, resulting in a total of 16 models. Therefore, to efficiently select our variables in the training dataset, we will apply the forward stepwise selection method. Once we have identified the optimal model in the training dataset, we will refit this model in the testing dataset.The forward selection process begins with the null model. At each step, a new variable is added to the current model, with the selection based on the greatest decrease in the residual sum of squares (RSS). Following this, we will identify the best models using a range of metrics for comprehensive evaluation.

The forward stepwise selection method is implemented by function `regsubsets()` from library `leaps`.Within this function, the argument `x` is set to `registered_users ~.`, specifying our model where registered_users is the response variable and all other variables are potential predictors. The argument `nvmax` is set to `4`, denoting the maximum number of variables to be considered in the variable selection process. The argument `method` then set to `forward`. 

Since the objective of our research question is inferential, we will use the adjusted R-squared to select the most effective generative model. This metric is preferred as it compares the fit of estimated models of different sizes. Thus, we use  `which.max`  function to find the model with the maximum adjusted R-squared value from all the possible models. The `coef` function is then used to extract the corresponding variables of this model. After initially fitting the model using the selected variables in the training dataset, we proceed to refit this same model in the testing dataset for further inference. 


#### Assumption:
1. Linear: The relationship between the explanatory variables and the response variable should be linear.
2. errors are independent
3. conditional distribution of error terms is normal
4. equal variance of error term
5. No multicollinearity: No explanatory variables are correlated 

#### Advantages:
1. use adjusted R squared as the evaluation metric, instead of R squared. This choice addresses a critical limitation of R-squared: its value increases with the addition of more variables to the model, due to the decrease in the residual sum of squares (RSS), regardless of whether these new variables significantly contribute to the model. Such a characteristic renders R-squared unsuitable for comparing nested models. The adjusted R-squared, on the other hand, is a refined version of R-squared that accounts for the number of predictors in the model, offering a more accurate assessment for our purposes.

2. forward selection is easy to apply, objective and reproducible.Additionally, it can enhance the generalizability of the model.
3. divide the dataset into training and testing sets. The training dataset will be used for selecting variables for the regression model, while the testing dataset will be reserved for inference purposes. This division is employed to prevent an increase in Type 1 error, which is a risk when the same data sample is used for both variable selection and model fitting. 


#### Limitation:
One limitation of this method is the potential to miss the optimal model, as not all possible combinations of predictors are evaluated during the selection process. This is because the inclusion of a new variable might render an existing variable in the model non-significant. However, once a variable is included in the model, it cannot be removed, which might limit the ability to identify the truly optimal set of predictors.

## 6. Implementation of a proposed model

In [6]:
bike_dat_step4$season <- as.factor(bike_dat_step4$season)
#bike_dat_step4$weathersit <- as.factor(bike_dat_step4$weathersit)
bike_dat_step4<-dummy_cols(bike_dat_step4, select_columns = c("season"),remove_first_dummy = TRUE,remove_selected_columns = TRUE)

head(bike_dat_step4)

,year,holiday,workingday,feeling_temp,humidity,windspeed,registered_users,season_2,season_3,season_4
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>
1,0,0,0,0.363625,0.805833,0.1604460,654,0,0,0
2,0,0,0,0.353739,0.696087,0.2485390,670,0,0,0
3,0,0,1,0.189405,0.437273,0.2483090,1229,0,0,0
4,0,0,1,0.212122,0.590435,0.1602960,1454,0,0,0
5,0,0,1,0.229270,0.436957,0.1869000,1518,0,0,0
6,0,0,1,0.233209,0.518261,0.0895652,1518,0,0,0


In [7]:
set.seed(98005)
bike_split <- initial_split(bike_dat_step4, prop = 0.5, strata = registered_users)
training_df <- training(bike_split)
testing_df <- testing(bike_split)

In [10]:
bike_sel <- regsubsets(
  x = registered_users ~., nvmax = 9,
  data = training_df,
  method = "forward",
)

In [11]:

bike_forward_summary <- summary(bike_sel)
bike_forward_summary_df <- tibble(
    n_input_variables = 1:9,
    RSQ = bike_forward_summary$rsq,
    RSS = bike_forward_summary$rss,
    ADJ.R2 = bike_forward_summary$adjr2,
    Cp = bike_forward_summary$cp,
    BIC = bike_forward_summary$bic,
)

ar_max = which.max(bike_forward_summary$adjr2) 
selected_var <- names(coef(bike_sel, ar_max))[-1]

In [12]:
set.seed(98005)
training_subset <- training_df %>% select(selected_var,registered_users)
testing_subset <- testing_df %>% select(selected_var,registered_users)
bike_ls <- lm(registered_users ~ .,data = training_subset)
csm <- update(bike_ls, registered_users~., data = testing_subset)

summary(csm)

Warning message:
“Using an external vector in selections was deprecated in tidyselect 1.1.0.
ℹ Please use `all_of()` or `any_of()` instead.
  # Was:
  data %>% select(selected_var)

  # Now:
  data %>% select(all_of(selected_var))

See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.”



Call:
lm(formula = registered_users ~ year + holiday + workingday + 
    feeling_temp + humidity + windspeed + season_2 + season_3 + 
    season_4, data = testing_subset)

Residuals:
     Min       1Q   Median       3Q      Max 
-2681.77  -345.14    73.67   411.58  1752.45 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)   1329.50     247.25   5.377 1.37e-07 ***
year          1725.44      71.91  23.994  < 2e-16 ***
holiday       -265.43     220.97  -1.201     0.23    
workingday    1005.79      79.50  12.651  < 2e-16 ***
feeling_temp  3885.56     364.32  10.665  < 2e-16 ***
humidity     -2221.37     267.40  -8.307 2.06e-15 ***
windspeed    -2076.14     476.34  -4.359 1.71e-05 ***
season_2       768.48     126.61   6.069 3.28e-09 ***
season_3       728.77     161.71   4.507 8.94e-06 ***
season_4      1399.96     108.62  12.889  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 664.2 on 357 degrees o

Through the process of forward selection, variables year, holiday, working day, feeling temperature, humidity, wind speed, and season were selected in our model. Despite the p-value for "holiday" exceeding the 5% significance threshold, it has been retained in the model based on the forward selection criteria, as other parameter coefficients demonstrate significance. The adjusted R-squared of the inference model is 0.8212, suggests that the model fits testing data very well. 